In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib
import ast

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_origin = pd.read_csv("chongluadao_dataset.csv")
df_train = pd.read_csv("chongluadao_dataset_process.csv")

In [5]:
inner = pd.merge(left=df_origin, right=df_train, left_on='url', right_on='urls',how='left')
del inner['urls']

In [6]:
inner.dropna(subset = ["features"], inplace=True)
inner

,labels,type,url,features
0,1,train,http://cranky-roentgen-f49ae1.netlify.app,[]
1,1,train,https://www.na-citiprepaidcard.com/,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,0,train,https://discreteconnection.com,[]
3,0,train,https://drrichswier.com,[]
4,0,train,https://attarax.com,[]
...,...,...,...,...
304402,1,val,http://eugnerally-wixsite-com.filesusr.com/htm...,"[4, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, ..."
304445,1,train,http://webmail.telephone-sfr.com/,[]
304539,1,val,https://deref-gmx.net/mail/client/THy7Wkvp09g/...,"[18, 0, 0, 1, 4, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,..."
304709,0,train,https://verisign.com,"[2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [7]:
index_names = inner[ inner['features'] == '[]' ].index
inner.drop(index_names, inplace = True)

In [72]:
pd.DataFrame(inner)['features'][1]

'[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]'

In [8]:
data_train = inner[inner['type'].str.contains("train")].dropna()
data_test  = inner[inner['type'].str.contains("test")].dropna()

In [9]:
data_train

,labels,type,url,features
1,1,train,https://www.na-citiprepaidcard.com/,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
5,0,train,https://sensortower.com,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,1,train,http://kovolem.cz/4458A73BC08A11997B7121510F00...,"[4, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, ..."
10,0,train,https://indexblogger.com,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
11,0,train,https://aamc.org,"[2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...,...
303869,1,train,https://biolinky.co/m4glacier,"[2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
304263,1,train,https://cutt.ly/LaPoste-colis,"[2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
304292,0,train,https://ijg.org,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
304349,1,train,http://holatoronto.com/demos/ad/login.php?cmd=...,"[11, 0, 0, 1, 3, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,..."


In [31]:
list(data_train['labels']).count(1)

4066

In [47]:
data_train['features'].values[0]

'[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]'

In [10]:
x = data_train['features'].values
y = data_train['labels'].values

# x = [np.array(i) for i in x]
x = [ast.literal_eval(i) for i in x]

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25, random_state =0 )

#----------------applying grid search to find best performing parameters 
from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [100, 700],
    'max_features': ['sqrt', 'log2'],
    'criterion' :['gini', 'entropy']}]

grid_search = GridSearchCV(RandomForestClassifier(),  parameters,cv =5, n_jobs= -1)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_features': ['sqrt', 'log2'],
                          'n_estimators': [100, 700]}])

In [33]:
print("Best Accurancy =" +str( grid_search.best_score_))
print("best parameters =" + str(grid_search.best_params_)) 

Best Accurancy =0.9894225186692897
best parameters ={'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100}


In [36]:
#fitting RandomForest regression with best params 
classifier = RandomForestClassifier(n_estimators = 100, criterion = "entropy", max_features = 'sqrt',  random_state = 0)
classifier.fit(x_train, y_train)

#predicting the tests set result
y_pred = classifier.predict(x_test)

#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

#pickle file joblib
joblib.dump(classifier, 'rf_final.pkl')

[[4898    8]
 [  53  997]]


['rf_final.pkl']

In [25]:
len(x[0]) == len(Features)

False

'Speical_Char'

In [39]:
Features = ['Speical_Char','Have_IP', 'Have_At','URL_length' ,'URL_Depth','Redirection', 'Time_get_redirect',
                        'port_in_url','use_http', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'https_Domain'
                        'Domain_lifespan', 'Domain_timeleft', 'same_asn','iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards','eval','unescape',
                        'escape', 'ActiveXObject','fromCharCode','atob','Punny_Code']

In [14]:
names = np.array(Features)
importances =classifier.feature_importances_
sorted_importances = sorted(importances, reverse=True)
indices = np.argsort(-importances)

In [15]:
pd.DataFrame(sorted_importances, names[indices], columns=['importance'])

IndexError: index 26 is out of bounds for axis 0 with size 26

In [20]:
names = np.array(Features)
importances =classifier.feature_importances_
sorted_importances = sorted(importances, reverse=True)
indices = np.argsort(-importances)
var_imp = pd.DataFrame(sorted_importances, names[indices], columns=['importance'])

IndexError: index 26 is out of bounds for axis 0 with size 26

In [28]:
fi_df = var_imp

fi_df.sort_values(by=['importance'], ascending=False,inplace=True)

plt.figure(figsize=(10,8))
sns.barplot(x=fi_df['importance'], y=names)
#Add chart labels
plt.title('RANDOM FOREST FEATURE IMPORTANCE')
plt.xlabel('FEATURE IMPORTANCE')
plt.ylabel('FEATURE NAMES')

NameError: name 'var_imp' is not defined

# Testing

In [37]:
from feature_extraction import *

classifier = joblib.load('rf_final.pkl')

ext = Extractor()
url = "https://github.com/numpy/numpy/issues/5303"

Vector = np.array(ext(url)).reshape(1, -1)

prediction = classifier.predict(Vector)
print(prediction)

https://github.com/numpy/numpy/issues/5303
[1]


In [142]:
import json

with open('Generate Dataset/dataset/blacklist','r') as f:
    black = json.load(f)

In [144]:
black[0]

{'_id': '604bb52f6a1161c2e7d8608c',
 'url': 'http://2021-lmht.gq/*',
 'type': 'scam',
 'level': 'high',
 'meta': {}}

In [ ]:
for i in black:
    Vector = np.array(ext(i['url'])).reshape(1, -1)
    
    prediction = classifier.predict(Vector )
    print(prediction)

Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
[0]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
[0]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
[0]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
[0]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0

[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1]
[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1]
Domain Age:  1826
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1]
Domain Age:  366
[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 3, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[0]
Domain Age:  5843
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1,

[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
[0]
Domain Age:  731
[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  731
[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[1]
Domain Age:  365
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1]
Domain Age:  1096
[0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
[0]
Domain Age:  1096
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1]
Error trying to connect to socket: closing socket
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1